## **Setting up Development Environment**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1


In [3]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

## **Importing Libraries**

In [4]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from transformers import AutoTokenizer, AutoModel

import pickle

## **Setting up Stemmer**

In [5]:
! rm -r tamil-stemmer-build
! rm input.txt
! rm output.txt

! mkdir tamil-stemmer-build
% cd tamil-stemmer-build
! git clone https://github.com/rdamodharan/tamil-stemmer.git
! tar -zxf tamil-stemmer/snowball-with-tamil.tgz
% cd snowball-with-tamil
! make

% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

rm: cannot remove 'input.txt': No such file or directory
rm: cannot remove 'output.txt': No such file or directory
/content/drive/My Drive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1/tamil-stemmer-build
Cloning into 'tamil-stemmer'...
remote: Enumerating objects: 88, done.
remote: Total 88 (delta 0), reused 0 (delta 0), pack-reused 88
Unpacking objects: 100% (88/88), done.
/content/drive/My Drive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1/tamil-stemmer-build/snowball-with-tamil
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/space.o compiler/space.c
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/tokeniser.o compiler/tokeniser.c
In file included from compiler/tokeniser.c:6:0:
compiler/tokeniser.c: In function ‘read_token’:
compiler/header.h:8:21: warning: this statement may fall through [-Wimplicit-fallthrough=]
 #define unless(C) if(!(C))
                     ^
comp

## **Loading Data**

In [6]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [7]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [8]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam1455,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [9]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [10]:
dataframe.shape

(5880, 3)

In [11]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [12]:
dataframe.shape

(5877, 3)

In [13]:
text = dataframe['text']

In [14]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [15]:
label = dataframe['category']

In [16]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [17]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [18]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

In [19]:
text = text.tolist()

! rm input.txt
! rm output.txt

for i in text:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

rm: cannot remove 'input.txt': No such file or directory
rm: cannot remove 'output.txt': No such file or directory


In [20]:
! ./tamil-stemmer-build/snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [21]:
text_file = open("output.txt", "r")

text = []

for line in text_file:
    text.append(line.strip())

In [22]:
# text = np.array(text)
# text

## **Feature Extraction**

In [23]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModel.from_pretrained("ai4bharat/indic-bert")

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.dense.weight', 'sop_classifier.classifier.weight', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.bias', 'predictions.bias', 'predictions.decoder.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [30]:
print(tokenized_input)

{'input_ids': tensor([[     2,   4436, 139013, 105466,   1385,  30231,  40291,  39555,  82156,
         150078, 194115,   5343, 104252,  82076,      3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [31]:
print(sample_output)

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.0101, -0.0098, -0.0160,  ..., -0.0302, -0.0208, -0.0186],
         [ 0.2039,  0.1569, -0.2342,  ..., -0.1102,  0.2575,  0.0765],
         [ 0.6726, -0.2458, -0.0159,  ...,  0.3238,  0.2636, -0.0796],
         ...,
         [ 0.9273, -0.2617, -0.1268,  ...,  0.2491,  0.1701,  0.0846],
         [ 0.5355, -0.0578, -0.0928,  ...,  0.2653,  0.0409, -0.5193],
         [-0.0101, -0.0098, -0.0160,  ..., -0.0302, -0.0208, -0.0186]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[-5.2381e-02,  5.5714e-02,  1.6412e-02,  6.1748e-04,  5.0623e-02,
          8.1442e-02,  4.2414e-02, -7.5105e-03, -1.2329e-02,  6.2801e-02,
         -1.0946e-02,  4.8848e-02,  1.6424e-02, -2.0096e-02,  4.9712e-02,
         -1.5399e-02, -4.3242e-02,  2.2940e-03, -1.2076e-01, -8.4439e-02,
          7.8570e-02,  7.2158e-02, -3.3552e-02, -9.9965e-02,  9.4612e-04,
         -3.3801e-02, -5.7639e-02, -6.0294e-03,  2.2355e-02,  7.2021e-03,
          2.

In [32]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [33]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [36]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./IndicBert-Embedded-TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./IndicBert-Embedded-TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./IndicBert-Embedded-TrainData/output1.pickle done
./IndicBert-Embedded-TrainData/output2.pickle done
./IndicBert-Embedded-TrainData/output3.pickle done
./IndicBert-Embedded-TrainData/output4.pickle done
./IndicBert-Embedded-TrainData/output5.pickle done
./IndicBert-Embedded-TrainData/output6.pickle done
./IndicBert-Embedded-TrainData/output7.pickle done
./IndicBert-Embedded-TrainData/output8.pickle done
./IndicBert-Embedded-TrainData/output9.pickle done
./IndicBert-Embedded-TrainData/output10.pickle done
./IndicBert-Embedded-TrainData/output11.pickle done
./IndicBert-Embedded-TrainData/output12.pickle done
./IndicBert-Embedded-TrainData/output13.pickle done
./IndicBert-Embedded-TrainData/output14.pickle done
./IndicBert-Embedded-TrainData/output15.pickle done
./IndicBert-Embedded-TrainData/output16.pickle done
./IndicBert-Embedded-TrainData/output17.pickle done
./IndicBert-Embedded-TrainData/output18.pickle done
./IndicBert-Embedded-TrainData/output19.pickle done
./IndicBert-Embedded-

In [18]:
# Loading Embedded Input Data from Disk

output = []

for i in range(30):
    file_name = "./IndicBert-Embedded-TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./IndicBert-Embedded-TrainData/output1.pickle done
./IndicBert-Embedded-TrainData/output2.pickle done
./IndicBert-Embedded-TrainData/output3.pickle done
./IndicBert-Embedded-TrainData/output4.pickle done
./IndicBert-Embedded-TrainData/output5.pickle done
./IndicBert-Embedded-TrainData/output6.pickle done
./IndicBert-Embedded-TrainData/output7.pickle done
./IndicBert-Embedded-TrainData/output8.pickle done
./IndicBert-Embedded-TrainData/output9.pickle done
./IndicBert-Embedded-TrainData/output10.pickle done
./IndicBert-Embedded-TrainData/output11.pickle done
./IndicBert-Embedded-TrainData/output12.pickle done
./IndicBert-Embedded-TrainData/output13.pickle done
./IndicBert-Embedded-TrainData/output14.pickle done
./IndicBert-Embedded-TrainData/output15.pickle done
./IndicBert-Embedded-TrainData/output16.pickle done
./IndicBert-Embedded-TrainData/output17.pickle done
./IndicBert-Embedded-TrainData/output18.pickle done
./IndicBert-Embedded-TrainData/output19.pickle done
./IndicBert-Embedded-

In [19]:
X = np.array(X)
X.shape

(5877, 1, 768)

In [20]:
X = X.reshape(5877, 768)
X.shape

(5877, 768)

In [21]:
y = label
y.shape

(5877,)

## **Train Test Split**

In [22]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
X_train.shape

(4701, 768)

In [24]:
X_val.shape

(1176, 768)

## **Logistic Regression**

In [25]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Support Vector Machine**

In [26]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Naive Bayes**

In [27]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.85      0.69      0.76       950
           1       0.27      0.50      0.35       226

    accuracy                           0.65      1176
   macro avg       0.56      0.59      0.56      1176
weighted avg       0.74      0.65      0.68      1176



## **Stochastic Gradient Descent**

In [28]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **K Nearest Neighbours**

In [29]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.82      0.93      0.87       950
           1       0.36      0.16      0.23       226

    accuracy                           0.78      1176
   macro avg       0.59      0.55      0.55      1176
weighted avg       0.74      0.78      0.75      1176



## **Decision tree**

In [30]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.84      0.81      0.82       950
           1       0.30      0.35      0.32       226

    accuracy                           0.72      1176
   macro avg       0.57      0.58      0.57      1176
weighted avg       0.73      0.72      0.73      1176



## **Random Forest**

In [31]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.82      0.99      0.90       950
           1       0.69      0.10      0.17       226

    accuracy                           0.82      1176
   macro avg       0.75      0.54      0.53      1176
weighted avg       0.80      0.82      0.76      1176



## **Majority Voting**

In [32]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Keeping only KNN, NB & DT in predictions due to poor performance
    predictions = [nb_pred[i], knn_pred[i], dt_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))    

              precision    recall  f1-score   support

           0       0.84      0.87      0.85       950
           1       0.35      0.29      0.32       226

    accuracy                           0.76      1176
   macro avg       0.59      0.58      0.58      1176
weighted avg       0.74      0.76      0.75      1176



## **Loading Testing Data**

In [33]:
dataframe_test = pd.read_csv(
    "Dataset/tam_offesive_withoutlabels_test.tsv", 
    sep='\t'
)
dataframe_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [34]:
dataframe_test.describe()

,id,text
count,654,654
unique,654,654
top,tam_test_325,௧டலூர் மாவட்டம் புவனகிரி வட்டம் தெற்க்குதிட்டை...
freq,1,1


In [35]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [36]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

In [37]:
text_test = text_test.tolist()

! rm input.txt
! rm output.txt

for i in text_test:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

rm: cannot remove 'input.txt': No such file or directory
rm: cannot remove 'output.txt': No such file or directory


In [38]:
! ./tamil-stemmer-build/snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [39]:
text_file = open("output.txt", "r")

text_test = []

for line in text_file:
    text_test.append(line.strip())

text_test = np.array(text_test)

## **Test Feature Extraction**

In [40]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModel.from_pretrained("ai4bharat/indic-bert")

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.bias', 'predictions.dense.weight', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [41]:
# Sample Output

tokenized_input = tokenizer(
        text_test[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [42]:
print(tokenized_input)

{'input_ids': tensor([[     2,  20264,   3048,  92072,  52721,   1385,   1385,  49940,   1657,
          32885,  13143, 124420,   3941,  10885,  20264,   3048,  34875,  15528,
              8,  41858,   6119,  14359,   3534,      3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [43]:
print(sample_output)

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.0220, -0.0103, -0.0184,  ..., -0.0277, -0.0151, -0.0177],
         [ 0.1349,  0.2533, -0.5680,  ..., -0.0561, -0.0241,  0.1819],
         [ 0.3869,  0.3930, -0.0643,  ...,  0.1082,  0.0555,  0.2692],
         ...,
         [ 0.5151,  0.1412, -0.3426,  ..., -0.0786,  0.3912,  0.5449],
         [ 0.6777, -0.0342,  0.0573,  ..., -0.0871,  0.1859,  0.0947],
         [-0.0220, -0.0103, -0.0184,  ..., -0.0277, -0.0151, -0.0177]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[-5.1660e-02,  5.3390e-02,  1.7966e-02, -4.3926e-03,  4.7348e-02,
          8.4199e-02,  3.7961e-02, -7.8548e-03, -1.4522e-02,  5.7318e-02,
         -9.9887e-03,  4.9222e-02,  1.3639e-02, -2.0439e-02,  5.5387e-02,
         -1.7512e-02, -4.5300e-02,  4.5599e-03, -1.1802e-01, -8.8100e-02,
          8.2153e-02,  7.8851e-02, -3.3273e-02, -9.7838e-02,  9.7679e-04,
         -3.3552e-02, -5.4551e-02, -7.7867e-03,  2.3278e-02,  1.1118e-02,
          2.

In [44]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [45]:
# Tokenizing Test Input Data

input = []

for i in text_test:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [46]:
# Embedding the Test Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Test Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./IndicBert-Embedded-TestData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./IndicBert-Embedded-TestData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./IndicBert-Embedded-TestData/output1.pickle done
./IndicBert-Embedded-TestData/output2.pickle done
./IndicBert-Embedded-TestData/output3.pickle done
./IndicBert-Embedded-TestData/output4.pickle done


In [47]:
# Loading Embedded Input Test Data from Disk

output = []

for i in range(4):
    file_name = "./IndicBert-Embedded-TestData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X_test = output
output = []

./IndicBert-Embedded-TestData/output1.pickle done
./IndicBert-Embedded-TestData/output2.pickle done
./IndicBert-Embedded-TestData/output3.pickle done
./IndicBert-Embedded-TestData/output4.pickle done


In [48]:
X_test = np.array(X_test)
X_test.shape

(654, 1, 768)

In [49]:
X_test = X_test.reshape(654, 768)
X_test.shape

(654, 768)

## **Getting Predictions**

In [50]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Support Vector Machine
svm_test_pred = svm.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

# Stochastic Gradient Descent
sgd_test_pred = sgd.predict(X_test)

# K Nearest Neighbours
knn_test_pred = knn.predict(X_test)

# Decision Tree
dt_test_pred = dt.predict(X_test)

# Random Forest
rf_test_pred = rf.predict(X_test)

In [51]:
# Majority Voting
mv_test_pred = []

for i in range(len(lr_test_pred)):
    one = 0
    zero = 0
    predictions = [nb_test_pred[i], knn_test_pred[i], dt_test_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_test_pred.append(1)
    else: mv_test_pred.append(0)

mv_test_pred = np.array(mv_test_pred)

## **Evaluating the Predictions**

In [52]:
dataframe_eval = pd.read_csv(
    "Dataset/tamil_offensive_test task 1 with labels.tsv", 
    sep='\t'
)

In [53]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [54]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred))

Logistic Regression
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [55]:
print('Support Vector Machine')
print(classification_report(y_test, svm_test_pred))

Support Vector Machine
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

Naive Bayes
              precision    recall  f1-score   support

           0       0.90      0.71      0.80       536
           1       0.33      0.66      0.44       118

    accuracy                           0.70       654
   macro avg       0.62      0.69      0.62       654
weighted avg       0.80      0.70      0.73       654



In [57]:
print('Stochastic Gradient Descent')
print(classification_report(y_test, sgd_test_pred))

Stochastic Gradient Descent
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
print('K Nearest Neighbours')
print(classification_report(y_test, knn_test_pred))

K Nearest Neighbours
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       536
           1       0.45      0.21      0.29       118

    accuracy                           0.81       654
   macro avg       0.65      0.58      0.59       654
weighted avg       0.77      0.81      0.78       654



In [59]:
print('Decision Tree')
print(classification_report(y_test, dt_test_pred))

Decision Tree
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       536
           1       0.30      0.34      0.32       118

    accuracy                           0.74       654
   macro avg       0.58      0.58      0.58       654
weighted avg       0.75      0.74      0.75       654



In [60]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred))

Random Forest
              precision    recall  f1-score   support

           0       0.83      0.99      0.91       536
           1       0.68      0.11      0.19       118

    accuracy                           0.83       654
   macro avg       0.76      0.55      0.55       654
weighted avg       0.81      0.83      0.78       654



In [61]:
print('Majority Voting')
print(classification_report(y_test, mv_test_pred))

Majority Voting
              precision    recall  f1-score   support

           0       0.88      0.89      0.88       536
           1       0.47      0.43      0.45       118

    accuracy                           0.81       654
   macro avg       0.67      0.66      0.67       654
weighted avg       0.80      0.81      0.81       654



## **Saving the Predictions**

In [62]:
# Logistic Regression

predictions = le.inverse_transform(lr_test_pred)

dataframe_test_lr = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_lr = pd.DataFrame(dataframe_test_lr)
dataframe_test_lr.to_csv("Output/IndicBert-LogisticRegression.csv")
dataframe_test_lr

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [63]:
# Support Vector Machine

predictions = le.inverse_transform(svm_test_pred)

dataframe_test_svm = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_svm = pd.DataFrame(dataframe_test_svm)
dataframe_test_svm.to_csv("Output/IndicBert-SupportVectorMachine.csv")
dataframe_test_svm

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [64]:
# Naive Bayes

predictions = le.inverse_transform(nb_test_pred)

dataframe_test_nb = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_nb = pd.DataFrame(dataframe_test_nb)
dataframe_test_nb.to_csv("Output/IndicBert-NaiveBayes.csv")
dataframe_test_nb

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,OFF
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,OFF
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",OFF
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [65]:
# Stochastic Gradient Descent

predictions = le.inverse_transform(sgd_test_pred)

dataframe_test_sgd = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_sgd = pd.DataFrame(dataframe_test_sgd)
dataframe_test_sgd.to_csv("Output/IndicBert-StochasticGradientDescent.csv")
dataframe_test_sgd

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [66]:
# K Nearest Neighbours

predictions = le.inverse_transform(knn_test_pred)

dataframe_test_knn = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_knn = pd.DataFrame(dataframe_test_knn)
dataframe_test_knn.to_csv("Output/IndicBert-KNearestNeighbours.csv")
dataframe_test_knn

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,OFF
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,OFF
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [67]:
# Decision Tree

predictions = le.inverse_transform(dt_test_pred)

dataframe_test_dt = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_dt = pd.DataFrame(dataframe_test_dt)
dataframe_test_dt.to_csv("Output/IndicBert-DecisionTree.csv")
dataframe_test_dt

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,OFF
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,OFF
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,OFF
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [68]:
# Random Forest

predictions = le.inverse_transform(rf_test_pred)

dataframe_test_rf = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_rf = pd.DataFrame(dataframe_test_rf)
dataframe_test_rf.to_csv("Output/IndicBert-RandomForest.csv")
dataframe_test_rf

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [69]:
# Majority Voting

predictions = le.inverse_transform(mv_test_pred)

dataframe_test_mv = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_mv = pd.DataFrame(dataframe_test_mv)
dataframe_test_mv.to_csv("Output/IndicBert-MajorityVoting.csv")
dataframe_test_mv

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,OFF
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,OFF
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT
